In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import time
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_absolute_error
import gzip
import json

In [2]:
import surprise

In [3]:
def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield json.loads(l)

In [4]:
import pandas as pd
import gzip

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df1 = getDF(r'C:\Users\Kani\Downloads\Grocery_and_Gourmet_Food.json.gz')

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split 

subset_80, subset_20 = train_test_split(df1, test_size=0.05, stratify=df1['overall'])

In [11]:
df1

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5.0,True,"06 4, 2013",ALP49FBWT4I7V,1888861614,Lori,Very pleased with my purchase. Looks exactly l...,Love it,1370304000,NaN,NaN,NaN
1,4.0,True,"05 23, 2014",A1KPIZOCLB9FZ8,1888861614,BK Shopper,Very nicely crafted but too small. Am going to...,Nice but small,1400803200,NaN,NaN,NaN
2,4.0,True,"05 9, 2014",A2W0FA06IYAYQE,1888861614,daninethequeen,still very pretty and well made...i am super p...,"the ""s"" looks like a 5, kina",1399593600,NaN,NaN,NaN
3,5.0,True,"04 20, 2014",A2PTZTCH2QUYBC,1888861614,Tammara,"I got this for our wedding cake, and it was ev...",Would recommend this to a friend!,1397952000,NaN,NaN,NaN
4,4.0,True,"04 16, 2014",A2VNHGJ59N4Z90,1888861614,LaQuinta Alexander,It was just what I want to put at the top of m...,Topper,1397606400,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5074155,5.0,True,"10 4, 2018",AGOPF2VTEOP57,B01HJHSVG6,Bob,"Love this product, very fresh, complete full f...",Exceclent product,1538611200,NaN,NaN,{'Size:': ' 2.29 oz'}
5074156,3.0,True,"10 3, 2018",A5X5TI4JCH9CN,B01HJHSVG6,Jacqueline Schafer,Good taste but small quantity for the expensiv...,Good taste but small quantity for the price us...,1538524800,NaN,NaN,{'Size:': ' 2.29 oz'}
5074157,4.0,True,"10 2, 2018",A1WMQQLC3TVAFI,B01HJHSVG6,Motionjunkie,Unlike some of the bad reviews regarding aroma...,See Comments below,1538438400,NaN,NaN,{'Size:': ' 2.29 oz'}
5074158,5.0,True,"09 30, 2018",ALVP6JZRTEDY6,B01HJHSVG6,Marissa,I absolutely love this vanilla bean paste. I h...,Love!!,1538265600,NaN,NaN,{'Size:': ' 2.29 oz'}


In [12]:
subset_20

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
4439786,3.0,True,"01 5, 2015",A1X5WMFVT4Z2O6,B00BWW7UTO,c.germann,Bottles arrived covered in sticky dried syrup....,Sticky mess.,1420416000,NaN,NaN,{'Size:': ' 50ml 3 Pack'}
857053,4.0,True,"09 18, 2013",ABGKBHY6NOIC1,B001650VTI,gerry virostek,"For many years I was able to find the ""Sugar F...","I am ""hooked"" on Chai Tea Latte!",1379462400,NaN,NaN,NaN
3526797,5.0,False,"10 21, 2015",A1EARN5PUVIF1S,B014UAGGHG,Jerry P. Danzig,Dark chocolate lovers will find a lot to like ...,Dark Chocolate for Adults,1445385600,NaN,NaN,NaN
3077137,5.0,True,"09 1, 2017",AVYH6VIOH5TYA,B00N0E8UO6,AH,great flavors,Five Stars,1504224000,NaN,NaN,"{'Size:': ' 6.2 oz.', 'Flavor:': ' Assorted fr..."
2690646,4.0,True,"05 1, 2016",A1BW33W5MN3R99,B00GEG4276,Marie P.,I wish I could pick the flavors. Great price.,Great price.,1462060800,NaN,NaN,{'Flavor:': ' Black Tea Variety Pack'}
...,...,...,...,...,...,...,...,...,...,...,...,...
1683581,5.0,True,"07 17, 2017",AAWWEFQQDS470,B0055IRNAC,lch89,Great product for a great price. This is what...,Great product for a great price,1500249600,NaN,NaN,{'Size:': ' 4 Pound'}
155596,5.0,True,"06 16, 2014",A3L2FJ67TRDP70,B000BKEUDM,Amazon Customer,This is a very delicious product. I throw som...,YUMMY,1402876800,NaN,NaN,NaN
1107705,5.0,True,"01 14, 2014",A3V11P7PLBLIV6,B001O8KQP6,Nina,have purchased several colors for my daughters...,Daughter loves these,1389657600,NaN,NaN,NaN
255606,5.0,True,"11 3, 2012",A1CTIGRPSM1Q2O,B000EVQ0NQ,N. Lockwood,Puffed rice with fresh fruit - great healthy b...,great cereal,1351900800,3,NaN,{'Size:': ' 6 Ounce (Pack of 12)'}


In [13]:
subset_20.to_csv('amazon_grocery_subset.csv')